In [1]:
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import utils
from yieldcurve import YieldCurve
import cashflow as cf

In [2]:
portfolio, market = utils.load_data()
TODAY = utils.set_today()

In [3]:
volume = 75000
maturity = 36
rate = 0.01
bull = cf.amortization_schedule(rate, maturity, volume, payment_type="BULLET")
lin = cf.amortization_schedule(rate, maturity, volume, payment_type='LINEAR')
ann = cf.amortization_schedule(rate, maturity, volume, payment_type='ANNUITY')

In [4]:
plot = False
EUR01 = YieldCurve(curve_type='EUR01', today=TODAY)
EUR01.fit(market, plot=plot)
date_list = utils.date_range('2016-01-01', '2035-01-01', 10)
spot_yields = EUR01.get_spot_yields(date_list, plot=plot)
forward_yields = EUR01.get_forward_yields(date_list, plot=plot)

date_list1 = utils.date_range('2016-01-01', '2035-01-01', 20)
date_list2 = utils.date_range('2016-01-01', '2035-01-01',10)
date_list2.append(datetime(2036,1,1))
floating_yields = EUR01.get_floating_yields(repayment_dates = date_list2, repricing_dates = date_list1, plot=plot)

In [5]:
cf.asset_cashflow(portfolio, market, 3, TODAY)

,date,cashflow,interest,capital,remaining
0,2014-10-15,0.083,0.083,0.0,996.0
1,2014-11-15,0.083,0.083,0.0,996.0
2,2014-12-15,996.083,0.083,996.0,0.0


In [9]:
portfolio_cashflow = cf.portfolio_cashflow(portfolio, market, TODAY)
portfolio_cashflow

,date,cashflow,interest,capital,remaining
0,2014-10-01,930.03875,0.03875,930.0,0.0
1,2014-10-30,0.08190,0.08190,0.0,1404.0
2,2014-11-30,1404.08190,0.08190,1404.0,0.0
3,2014-10-15,0.08300,0.08300,0.0,996.0
4,2014-11-15,0.08300,0.08300,0.0,996.0
...,...,...,...,...,...
10335,2030-08-30,0.00000,-0.00000,0.0,-3930.0
10336,2030-09-30,0.00000,-0.00000,0.0,-3930.0
10337,2030-10-30,0.00000,-0.00000,0.0,-3930.0
10338,2030-11-30,0.00000,-0.00000,0.0,-3930.0
